# Generate indiv plots 
for short recordings from equivitals

In [ ]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport qex
%aimport eq
%aimport respy

In [ ]:
race_dets = pd.read_csv('./Data/Relay_dets.csv',sep = ',')
race_dets['Start time'] = pd.to_datetime('2023-05-13 ' + race_dets['Start time'] + ' +0200')
race_dets['End time'] = pd.to_datetime('2023-05-13 ' + race_dets['End time'] + ' +0200')
race_dets['Duration'] = (race_dets['End time'] - race_dets['Start time']).dt.total_seconds()

dev_dets = race_dets.loc[~race_dets['PartID'].isna(),:].reset_index(drop = True).copy()
dev_dets 

# Trim to race time data
Clean up extra files from qiosk and generate trimmed version of signals during the time interval of interest.

This process requires stepping through the participant files one at a time to check on data quality. for example, in this set, one expected participant didn't have a recording during race time. BR606 To be evaluated later. 

In [ ]:
partID = dev_dets.loc[0,'PartID']
print(partID)
projectName = ''
sep = '/'
path = './Data/'+ partID
df_datafiles = qex.qiosk_rec_check(path,projectName,sep)
df_datafiles

In [ ]:
s_files = df_datafiles.iloc[:-1,:].copy()
s_files

In [ ]:
# remove files
discardPath = './Data/waste/'
for i,row in s_files.iterrows():
    tic = time.time()
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,sep)  # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('/')[-1]
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                dout_f = discardPath + fileName
                shutil.move(fi,dout_f) # move the exported csv files to the discard folder
        if fileName.lower().endswith('sem'):
            out_f = discardPath + fileName
            shutil.move(fi,out_f)
            

In [ ]:
projectPath = './Data/Relay/'
# Interval to keep 
t1 = pd.to_datetime('2023-05-13 17:00:00.00+0200')# 17:15:00 + 0100  less 15 minutes
t2 = pd.to_datetime('2023-05-13 18:55:00.00+0200') # 18:39:13 + 0100 plus 15 minutes
#t2 = t1+pd.to_timedelta(240,'s')
[t1,t2]

In [ ]:
partID = dev_dets.loc[8,'PartID']
print(partID)
projectName = ''
sep = '/'
path = './Data/'+ partID
df_datafiles = qex.qiosk_rec_check(path,projectName,sep)
df_datafiles

In [ ]:
# test cutting
tic = time.time()
dataLoc = df_datafiles.iloc[0,14]
matched = qex.matched_files(dataLoc,path,sep) # outputs locations of csv 
for fi in matched:
    if fi.lower().endswith('.csv'):
    #if not fi.lower().endswith('.sem'):
        fp = fi.split(sep)
        fn = fp[-1].split('.')
        fn_new = fn[0]+'-trimmed.csv'
        print(fn_new)
        V = qex.cut_by_time(fi,t1,t2)
        print(V.columns)
        V.to_csv(projectPath + fn_new)
        print(time.time()-tic)

In [ ]:
fi

# quick plot group
Generate full interval and race interval plots for a single participant 

plot 
1. RR
2. Resp
3. QoM
4. Temp

In [ ]:
projectPath = './Data/Relay/'
# Interval to keep 
t1 = pd.to_datetime('2023-05-13 17:00:00.00+0200')# 17:15:00 + 0100  less 15 minutes
t2 = pd.to_datetime('2023-05-13 18:55:00.00+0200') # 18:39:13 + 0100 plus 15 minutes
#t2 = t1+pd.to_timedelta(240,'s')
[t1,t2]
eqfiles = os.listdir(projectPath)
eqfiles.sort()

In [ ]:
for partID in dev_dets['PartID']:
    for f in eqfiles:
        if partID in f: 
            if f.startswith('RR'):
                print(f)
                sf = 5
                fig, (axs) = plt.subplots(1, 1,figsize = [14,3])
                ax=axs#[0]
                V = pd.read_csv(projectPath + f)
                V['DateTime'] = pd.to_datetime(V['DateTime'])
                V = V.loc[V['Interbeat Interval (MS)']>200,:].reset_index(drop=True)
                sig_t = (V['DateTime']-V['DateTime'][0]).dt.total_seconds()
                sig_v = V['Interbeat Interval (MS)']

                time_s = np.round(np.linspace(0,sig_t.iloc[-1],int(sf*(sig_t.iloc[-1])),endpoint=False),3)
                time_dt = V['DateTime'][0] + pd.to_timedelta(time_s,unit='s')
                df_sig = pd.DataFrame(index = time_dt)
                ax.scatter(x =V['DateTime'],y=(60000/sig_v),color='r',alpha = 0.2,label='Single Beat')

                HR = (60000/sig_v).rolling(10,center=True).median() # really lazy smoothing
                f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                df_sig.loc[:,'10 bt Median'] = f(time_s)
                df_sig['10 bt Median'].plot(ax = ax,label = '10 bt Median')

                HR = (60000/sig_v).rolling(60,center=True).median() # really lazy smoothing
                f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                df_sig.loc[:,'60bMed'] = f(time_s)
                df_sig['60bMed'].plot(color = 'k',ax = ax,label = '60 bt Median')

                ax.set_yticks([40,60,80,100,120,140,160,180,200,220,240])
                ax.set_xticks(race_dets['Start time'])
                ax.set_xticklabels(race_dets['Start time'].dt.strftime('%H:%M'))
                ax.set_xlim([t1,t2])
                ax.set_title(partID)
                ax.grid()
                plt.show()           

In [ ]:
for partID in dev_dets['PartID']:
    for f in eqfiles:
        if partID in f: 
            if f.startswith('DATA'):
                print(f)
                sf = 5
                fig, (axs) = plt.subplots(1, 1,figsize = [14,3])
                ax=axs#[0]
                V = pd.read_csv(projectPath + f)
                V['DateTime'] = pd.to_datetime(V['DateTime'])                
                sig_t = (V['DateTime']-V['DateTime'][0]).dt.total_seconds()
                time_dt = V['DateTime'][0] + pd.to_timedelta(sig_t,unit='s')
                
                df_sig = pd.DataFrame(index = time_dt)
                df_sig['Temp'] = V['TEMPERATURE'].values
                ax.scatter(x =V['DateTime'],y=V['TEMPERATURE'],color='b',alpha = 0.2)

#                 df_sig['Temp'].plot(color = 'k',ax = ax,label = 'Skin Tempurature') # pandas is ludicris
                ax.set_ylim([32,39])
                
                ax.set_xticks(race_dets['Start time'].dt.tz_convert('UCT'))
                ax.set_xticklabels(race_dets['Start time'].dt.strftime('%H:%M'))
                ax.set_xlim([t1,t2])
                ax.set_title(partID)
                ax.grid()
                plt.show()           

In [ ]:
sf = 25 # hz 
cutoff = 2 #cutoff = np.array([0.05,1]) 
nyq = 0.5 * sf 
order = 2 
normal_cutoff = cutoff / nyq
b, a = butter(order, normal_cutoff, btype='lowpass', analog=False)

Comps = ['Inspiration_Onset', 'Expiration_Onset', 'Inspiration_high',
       'Expiration_high','Post_Expiration']
windows = {'Inspiration_Onset':0.5, 'Expiration_Onset':0.5, 'Inspiration_high':0.2,
       'Expiration_high':0.2,'Post_Expiration':0.2}

scale_high=0.5
scale_low = 0.3

for partID in dev_dets['PartID']:
    for f in eqfiles:
        if partID in f: 
            if f.startswith('RESPACC'):
                print(f)
                sf = 25
                fig, (axs) = plt.subplots(1, 1,figsize = [14,3])
                ax=axs#[0]
                V = pd.read_csv(projectPath + f)
                V['DateTime'] = pd.to_datetime(V['DateTime'])                
                ax.scatter(x =V['DateTime'],y=V['Breathing'],color='k',alpha = 0.2)
                ax.set_xticks(race_dets['Start time'].dt.tz_convert('UCT'))
                ax.set_xticklabels(race_dets['Start time'].dt.strftime('%H:%M'))
                ax.set_xlim([t1,t2])
                ax.set_title(partID)
                ax.grid()
                plt.show()           

In [ ]:
sf = 25 # hz 
cutoff = 2 #cutoff = np.array([0.05,1]) 
nyq = 0.5 * sf 
order = 2 
normal_cutoff = cutoff / nyq
b, a = butter(order, normal_cutoff, btype='lowpass', analog=False)

Comps = ['Inspiration_Onset', 'Expiration_Onset', 'Inspiration_high',
       'Expiration_high','Post_Expiration']
windows = {'Inspiration_Onset':0.5, 'Expiration_Onset':0.5, 'Inspiration_high':0.2,
       'Expiration_high':0.2,'Post_Expiration':0.2}

scale_high=0.5
scale_low = 0.3

for partID in dev_dets['PartID']:
    for f in eqfiles:
        if partID in f: 
            if f.startswith('FASTACC'):
                print(f)
                sf = 25
                fig, (axs) = plt.subplots(1, 1,figsize = [14,3])
                ax=axs#[0]
                V = pd.read_csv(projectPath + f)
                V['DateTime'] = pd.to_datetime(V['DateTime']) 
                #         df = V.loc[:,[ 'Vert Accelerometer','Lat Accelerometer','Long Accelerometer']]

                V['Jerk'] = np.sqrt(np.square(V.loc[:,[ 'Vert Accelerometer','Lat Accelerometer','Long Accelerometer']].diff()).sum(axis=1))
                ax.scatter(x =V['DateTime'],y=V['Jerk'],color='c',alpha = 0.2)
                ax.set_xticks(race_dets['Start time'].dt.tz_convert('UCT'))
                ax.set_xticklabels(race_dets['Start time'].dt.strftime('%H:%M'))
                ax.set_ylim([0,6000])
                ax.set_xlim([t1,t2])
                ax.set_title(partID)
                ax.grid()
                plt.show()           

# Summary list of plots, full race



In [ ]:
race_times = list(race_dets['Start time'])
race_times.append(race_dets['End time'][14])
race_times =pd.Series(data=race_times)

In [ ]:
mydpi = 192

In [ ]:
for i in range(len(dev_dets)):
    partID = dev_dets.loc[i,'PartID'] 
    fig, (axs) = plt.subplots(4, 1,sharex = True, figsize=(10,7), dpi=mydpi) # 14 10
    for file in eqfiles:
        if partID in file: 
            if file.startswith('RR'):
                sf = 5
                ax=axs[0]
                V = pd.read_csv(projectPath + file)
                V['DateTime'] = pd.to_datetime(V['DateTime'])
                V = V.loc[V['Interbeat Interval (MS)']>200,:].reset_index(drop=True)
                sig_t = (V['DateTime']-V['DateTime'][0]).dt.total_seconds()
                sig_v = V['Interbeat Interval (MS)']
                ax.scatter(x =V['DateTime'],y=(60000/V['Interbeat Interval (MS)']),s=10, color='r',alpha = 0.2,label='Single Beat')

                time_s = np.round(np.linspace(0,sig_t.iloc[-1],int(sf*(sig_t.iloc[-1])),endpoint=False),3)
                time_dt = V['DateTime'][0] + pd.to_timedelta(time_s,unit='s')
                df_sig = pd.DataFrame(index = time_dt)
                HR = (60000/sig_v).rolling(10,center=True).median() # really lazy smoothing
                f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                df_sig.loc[:,'10 bt Median'] = f(time_s)
                HR = (60000/sig_v).rolling(60,center=True).median() # really lazy smoothing
                f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                df_sig.loc[:,'60bMed'] = f(time_s)
                df_sig['10 bt Median'].plot(ax = ax,label = '10 beat Median')
                df_sig['60bMed'].plot(color = 'k',ax = ax,label = '60 beat Median')

                ax.set_yticks([40,60,80,100,120,140,160,180,200,220,240])
                ax.set_yticklabels(['40','','80','','120','','160','','200','','240'])
                ax.set_ylim([40,240])
                ax.set_ylabel('Heart BPM')
                ax.legend(fontsize = 8)
                ax.grid()
            if file.startswith('RESPACC'):
                sf = 25
                ax=axs[1]
                V = pd.read_csv(projectPath + file)
                V['DateTime'] = pd.to_datetime(V['DateTime'])                
                ax.scatter(x =V['DateTime'],y=V['Breathing'],s=5,color='k',alpha = 0.2)
                ax.set_ylabel('Resp Depth')
                ax.grid()
            if file.startswith('FASTACC'):               
                ax=axs[2]
                V = pd.read_csv(projectPath +file)
                V['DateTime'] = pd.to_datetime(V['DateTime']) 
                V['Jerk'] = np.sqrt(np.square(V.loc[:,[ 'Vert Accelerometer','Lat Accelerometer','Long Accelerometer']].diff()).sum(axis=1))
                ax.scatter(x =V['DateTime'],y=V['Jerk'],s=3,color='c',alpha = 0.2)
                ax.set_ylim([0,6000])
                ax.set_ylabel('QoM (jerk)')
                ax.grid()
            if file.startswith('DATA'):
                ax=axs[3]
                V = pd.read_csv(projectPath + file)
                V['DateTime'] = pd.to_datetime(V['DateTime'])                
                ax.scatter(x =V['DateTime'],y=V['TEMPERATURE'],color='b',alpha = 0.2)
                ax.set_ylim([32,39])
                ax.set_ylabel('Skin Temp C')
                ax.grid()
            axs[3].set_xticks(race_times)
            axs[3].set_xticklabels(race_times.dt.strftime('%H:%M:%S'),fontsize = 8,rotation = 45)
            axs[3].set_xlim([t1,t2])
            axs[3].set_xlabel('Relay points')
            axs[0].set_title(dev_dets.loc[i,'Participant'] + ' Full race time Equivital readings')

    plt.savefig('./Plots/Relay/'+ partID+ '_Full_Relay.png',bbox_inches = 'tight',dpi=mydpi)
    plt.show()  

In [ ]:
dev_dets['PartID']

In [ ]:

buff = 120 # 2 minutes

for i in range(len(dev_dets)):
#i = 0
    partID = dev_dets.loc[i,'PartID']

    ts = dev_dets.loc[i,'Start time']-pd.to_timedelta(buff,unit='s')
    te = dev_dets.loc[i,'End time']+pd.to_timedelta(2*buff,unit='s')
    print([ts,te])
    fig, (axs) = plt.subplots(4, 1,sharex = True, figsize=(12,10), dpi=mydpi)
    #fig, (axs) = plt.subplots(4, 1,sharex = True, figsize = [12,10])

    for file in eqfiles:
        if partID in file: 
            if file.startswith('RR'):
                sf = 5
                ax=axs[0]
                V = qex.cut_by_time(projectPath + file,ts,te)
    #             V = pd.read_csv(projectPath + file)
                V['DateTime'] = V.index #pd.to_datetime(V.Index)
                V = V.loc[V['Interbeat Interval (MS)']>200,:].reset_index(drop=True)
                sig_t = (V['DateTime']-V['DateTime'][0]).dt.total_seconds()
                sig_v = V['Interbeat Interval (MS)']
                ax.scatter(x =V['DateTime'],y=(60000/V['Interbeat Interval (MS)']),s=10,color='r',alpha = 0.2,label='Single Beat')

                time_s = np.round(np.linspace(0,sig_t.iloc[-1],int(sf*(sig_t.iloc[-1])),endpoint=False),3)
                time_dt = V['DateTime'][0] + pd.to_timedelta(time_s,unit='s')
                df_sig = pd.DataFrame(index = time_dt)
                HR = (60000/sig_v).rolling(10,center=True).median() # really lazy smoothing
                f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                df_sig.loc[:,'10 bt Median'] = f(time_s)
                HR = (60000/sig_v).rolling(60,center=True).median() # really lazy smoothing
                f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                df_sig.loc[:,'60bMed'] = f(time_s)
                df_sig['10 bt Median'].plot(ax = ax,label = '10 bt Median')
                df_sig['60bMed'].plot(color = 'k',ax = ax,label = '60 bt Median')

                ax.set_yticks([40,60,80,100,120,140,160,180,200,220,240])
                ax.set_ylim([40,240])
                ax.set_ylabel('Heart BPM')
                ax.legend()
                ax.grid()
            if file.startswith('RESPACC'):
                sf = 25
                ax=axs[1]
                V = qex.cut_by_time(projectPath + file,ts,te) #V = pd.read_csv(projectPath + file)
                V['Breathing'].plot(color='k',ax=ax)
                ax.set_ylabel('Resp Depth')
                ax.grid()
            if file.startswith('FASTACC'):               
                ax=axs[2]
                V = qex.cut_by_time(projectPath + file,ts,te) #V = pd.read_csv(projectPath +file)
                V['Jerk'] = np.sqrt(np.square(V.loc[:,[ 'Vert Accelerometer','Lat Accelerometer','Long Accelerometer']].diff()).sum(axis=1))
                V['Jerk'].plot(color='c',ax=ax)
                ax.set_ylim([0,6000])
                ax.set_ylabel('QoM (jerk)')
                ax.grid()
            if file.startswith('DATA'):
                ax=axs[3]
                V = qex.cut_by_time(projectPath + file,ts,te) #V = pd.read_csv(projectPath + file)
                V['DateTime'] = V.index #pd.to_datetime(V.Index)                
                ax.scatter(x =V['DateTime'],y=V['TEMPERATURE'],color='b',alpha = 0.2)
                ax.set_ylim([32,39])
                ax.set_ylabel('Skin Temp C')
                ax.grid()
            axs[3].set_xticks(race_times)
            axs[3].set_xticklabels(race_times.dt.strftime('%H:%M:%S'),rotation = 10)
            axs[3].set_xlim([ts,te])
            axs[3].set_xlabel('Relay points')
            axs[0].set_title(' '.join([dev_dets.loc[i,'Participant'],'Ettape',str(dev_dets.loc[i,'Leg']),dev_dets.loc[i,'Point'], 'Equivital readings']))

    plt.savefig('./Plots/Relay/'+ partID+ '_leg_Relay.png',bbox_inches = 'tight',dpi=mydpi)
    plt.show()  

In [ ]:
dev_dets.columns

# pre, mid, post
excerpt 30 s pre, mid run and post run to contrast

In [ ]:
buff = 5 # 2 minutes

for i in range(len(dev_dets)):
#     i = 0
    partID = dev_dets.loc[i,'PartID']
    seg_ts = []
    seg_ts.append(dev_dets.loc[i,'Start time']-pd.to_timedelta(120,unit='s'))
    seg_ts.append(dev_dets.loc[i,'Start time'] + 0.5*(dev_dets.loc[i,'End time']-dev_dets.loc[i,'Start time'])-pd.to_timedelta(10,unit='s'))
    seg_ts.append(dev_dets.loc[i,'End time']+pd.to_timedelta(120,unit='s'))
    seg_i = 0
    ts = seg_ts[seg_i]
    te = seg_ts[seg_i]+pd.to_timedelta(buff,unit='s')
    print([ts,te])

    fig, (axs) = plt.subplots(4, 1,sharex = True, figsize=(12,10), dpi=mydpi)
    #fig, (axs) = plt.subplots(4, 3, figsize = [12,10])

    for file in eqfiles:
        if partID in file: 
            if file.startswith('RR'):
                for seg_i in range(len(seg_ts)):
                    ts = seg_ts[seg_i]
                    te = seg_ts[seg_i]+pd.to_timedelta(buff,unit='s')
                    sf = 5
                    ax=axs[0,seg_i]
                    V = qex.cut_by_time(projectPath + file,ts,te)
        #             V = pd.read_csv(projectPath + file)
                    V['DateTime'] = V.index #pd.to_datetime(V.Index)
                    V = V.loc[V['Interbeat Interval (MS)']>200,:].reset_index(drop=True)
                    sig_t = (V['DateTime']-V['DateTime'][0]).dt.total_seconds()
                    sig_v = V['Interbeat Interval (MS)']
                    ax.scatter(x =V['DateTime'],y=(60000/V['Interbeat Interval (MS)']),s=10,color='r',alpha = 0.2,label='Single Beat')

                    time_s = np.round(np.linspace(0,sig_t.iloc[-1],int(sf*(sig_t.iloc[-1])),endpoint=False),3)
                    time_dt = V['DateTime'][0] + pd.to_timedelta(time_s,unit='s')
                    df_sig = pd.DataFrame(index = time_dt)
                    HR = (60000/sig_v).rolling(10,center=True).median() # really lazy smoothing
                    f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                    df_sig.loc[:,'10 bt Median'] = f(time_s)
                    HR = (60000/sig_v).rolling(60,center=True).median() # really lazy smoothing
                    f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                    df_sig.loc[:,'60bMed'] = f(time_s)
                    df_sig['10 bt Median'].plot(ax = ax,label = '10 bt Median')
                    df_sig['60bMed'].plot(color = 'k',ax = ax,label = '60 bt Median')
                    
                    ax.set_yticks([40,60,80,100,120,140,160,180,200,220,240])
                    ax.set_ylim([40,240])
                    ax.set_xticklabels([])
                    ax.set_xlim([ts,te])
                    ax.grid()
                axs[0,0].set_ylabel('Heart BPM')
            if file.startswith('RESPACC'):
                for seg_i in range(len(seg_ts)):
                    ts = seg_ts[seg_i]
                    te = seg_ts[seg_i]+pd.to_timedelta(buff,unit='s')
                    ax=axs[1,seg_i]
                    V = qex.cut_by_time(projectPath + file,ts,te) #V = pd.read_csv(projectPath + file)
                    V['Breathing'].plot(color='k',ax=ax)
                    ax.set_xticklabels([])
                    ax.set_xlim([ts,te])
                    ax.grid()
                axs[1,0].set_ylabel('Resp Depth')
            if file.startswith('FASTACC'):               
                for seg_i in range(len(seg_ts)):
                    ts = seg_ts[seg_i]
                    te = seg_ts[seg_i]+pd.to_timedelta(buff,unit='s')
                    ax=axs[2,seg_i]
                    V = qex.cut_by_time(projectPath + file,ts,te) #V = pd.read_csv(projectPath +file)
                    V['Jerk'] = np.sqrt(np.square(V.loc[:,[ 'Vert Accelerometer','Lat Accelerometer','Long Accelerometer']].diff()).sum(axis=1))
                    V['Jerk'].plot(color='c',ax=ax)
#                     ax.set_ylim([0,6000])
                    ax.set_xlim([ts,te])
                    ax.set_xticklabels([])
                    ax.grid()
                axs[2,0].set_ylabel('QoM (jerk)')
            if file.startswith('ECG'):
                for seg_i in range(len(seg_ts)):
                    ts = seg_ts[seg_i]
                    te = seg_ts[seg_i]+pd.to_timedelta(buff,unit='s')
                    ax=axs[3,seg_i]
                    V = qex.cut_by_time(projectPath + file,ts,te) #V = pd.read_csv(projectPath + file)
                    V['DateTime'] = V.index #pd.to_datetime(V.Index)      
                    V['Lead 1'].plot(color='b',ax=ax)
                    ax.set_xlim([ts,te])
                    ax.grid()
                    ax.set_ylim([-2,2])
                axs[3,0].set_ylabel('ECG')
    axs[0,1].set_title(' '.join([dev_dets.loc[i,'Participant'],str(buff),'s samples','Equivital readings']))
    axs[3,0].set_xlabel('Pre-Run (from -120s)')
    axs[3,1].set_xlabel('Mid-Run')
    axs[3,2].set_xlabel('Post-Run (from +120s)')
    plt.savefig('./Plots/Relay/'+ partID+ '_5s_samples_Relay.png',bbox_inches = 'tight',dpi=mydpi)
    plt.show()  

# Summary resp style

In [ ]:
projectPath = './Data/Relay/'
# Interval to keep 
t1 = pd.to_datetime('2023-05-13 17:00:00.00+0200').tz_convert('UTC')# 17:15:00 + 0100  less 15 minutes
t2 = pd.to_datetime('2023-05-13 18:55:00.00+0200').tz_convert('UTC') # 18:39:13 + 0100 plus 15 minutes
#t2 = t1+pd.to_timedelta(240,'s')
[t1,t2]

In [ ]:
dev_dets

In [ ]:
# phase plots only worth looking at much more closely. 
# tryin again when cutting to leg

sf = 25 # hz 
cutoff = 2 #cutoff = np.array([0.05,1]) 
nyq = 0.5 * sf 
order = 2 
normal_cutoff = cutoff / nyq
b, a = butter(order, normal_cutoff, btype='lowpass', analog=False)

# Comps = ['Inspiration_Onset', 'Expiration_Onset', 'Inspiration_high',
#        'Expiration_high','Post_Expiration']
# windows = {'Inspiration_Onset':0.5, 'Expiration_Onset':0.5, 'Inspiration_high':0.2,
#        'Expiration_high':0.2,'Post_Expiration':0.2}

insp_t_bin = np.linspace(0,3,int(1.5*sf)+1,endpoint=True)
depth_bin = np.linspace(0,1000,int(1.5*sf)+1,endpoint=True)
scale_high=0.5
scale_low = 0.3
for i in range(len(dev_dets)):
    partID = dev_dets.loc[i,'PartID']
    buff = 10
    ts = (dev_dets.loc[i,'Start time']-pd.to_timedelta(buff,unit='s')).tz_convert('UTC')
    te = (dev_dets.loc[i,'End time']+pd.to_timedelta(buff,unit='s')).tz_convert('UTC')
    print([ts,te])

    for file in eqfiles:
        if partID in  file: 
            if file.startswith('RESPACC'):
                print(file)
                V = qex.cut_by_time(projectPath + file,t1,t2)
                t1 = V.index[0]
                V['DateTime'] = V.index 
                sig_t = (V['DateTime']-V['DateTime'][0]).dt.total_seconds()
                sig_v = V['Breathing']
                f = interpolate.interp1d(sig_t, sig_v,fill_value='extrapolate')

                dur = sig_t.max()
                time_s = np.round(np.linspace(0,dur,int(sf*(dur)),endpoint=False),3)
                df_sig = pd.DataFrame(index = time_s)
                df_sig['Resp'] = filtfilt(b, a, f(time_s))

                Breaths = respy.Breath_Features(df_sig['Resp'],filtered=True)  

                fig, axs = plt.subplots(2, 3,figsize = [12,6])

                Q_Breaths = Breaths.loc[Breaths['In']<(ts-t1).total_seconds(),:]
                Q_Breaths = Q_Breaths.loc[Q_Breaths['In']>((ts-t1).total_seconds()-600),:]
                ax = axs[0,0]
                counts, xedges, yedges, im = ax.hist2d(Q_Breaths.Insp_T,Q_Breaths.Depth, bins=(insp_t_bin,depth_bin), cmap=plt.cm.jet)
                fig.colorbar(im, ax=ax)
                ax.set_title('Pre run (10 minutes)')
                ax.set_xlabel('Inspiration Time (s)')
                ax.set_ylabel('Depth') 
                ax = axs[1,0]
                counts, xedges, yedges, im = ax.hist2d(Q_Breaths.Exp_T,Q_Breaths.Depth, bins=(insp_t_bin,depth_bin), cmap=plt.cm.jet)
                fig.colorbar(im, ax=ax)
                ax.set_xlabel('Expiration Time (s)')
                ax.set_ylabel('Depth') 


                Q_Breaths = Breaths.loc[Breaths['In']<(te-t1).total_seconds(),:]
                Q_Breaths = Q_Breaths.loc[Q_Breaths['In']>(ts-t1).total_seconds(),:]
                ax = axs[0,1]
                counts, xedges, yedges, im = ax.hist2d(Q_Breaths.Insp_T,Q_Breaths.Depth, bins=(insp_t_bin,depth_bin), cmap=plt.cm.jet)
                fig.colorbar(im, ax=ax)
                ax.set_title('During run ' + partID)
                ax.set_xlabel('Inspiration Time (s)')
                ax = axs[1,1]
                counts, xedges, yedges, im = ax.hist2d(Q_Breaths.Exp_T,Q_Breaths.Depth, bins=(insp_t_bin,depth_bin), cmap=plt.cm.jet)
                fig.colorbar(im, ax=ax)
                ax.set_xlabel('Expiration Time (s)')

                ax = axs[0,2]
                Q_Breaths = Breaths.loc[Breaths['In']>(te-t1).total_seconds(),:]
                Q_Breaths = Q_Breaths.loc[Q_Breaths['In']<((te-t1).total_seconds()+600),:]
                counts, xedges, yedges, im = ax.hist2d(Q_Breaths.Insp_T,Q_Breaths.Depth, bins=(insp_t_bin,depth_bin), cmap=plt.cm.jet)
                fig.colorbar(im, ax=ax)
                ax.set_title('Post run (10 minutes)')
                ax.set_xlabel('Inspiration Time (s)')
                ax = axs[1,2]
                counts, xedges, yedges, im = ax.hist2d(Q_Breaths.Exp_T,Q_Breaths.Depth, bins=(insp_t_bin,depth_bin), cmap=plt.cm.jet)
                fig.colorbar(im, ax=ax)
                ax.set_xlabel('Expiration Time (s)')

    plt.savefig('./Plots/Relay/'+ partID+ '_resp_phases.png',dpi = 300)

    plt.show()           

# remaining

In [ ]:
sf = 5
fig, (axs) = plt.subplots(1, 1,figsize = [14,5])
ax=axs#[0]

for i in range(len(dev_dets)):
    partID = dev_dets.loc[i,'PartID']
    for f in eqfiles:
        if partID in f: 
            if f.startswith('RR'):
                print(f)

                V = pd.read_csv(projectPath + f)
                V['DateTime'] = pd.to_datetime(V['DateTime'])
                V = V.loc[V['Interbeat Interval (MS)']>200,:].reset_index(drop=True)
                sig_t = (V['DateTime']-V['DateTime'][0]).dt.total_seconds()
                sig_v = V['Interbeat Interval (MS)']

                time_s = np.round(np.linspace(0,sig_t.iloc[-1],int(sf*(sig_t.iloc[-1])),endpoint=False),3)
                time_dt = V['DateTime'][0] + pd.to_timedelta(time_s,unit='s')
                df_sig = pd.DataFrame(index = time_dt)

                HR = (60000/sig_v).rolling(60,center=True).median() # really lazy smoothing
                f = interpolate.interp1d(sig_t, HR,fill_value='extrapolate')
                df_sig.loc[:,'60bMed'] = f(time_s)
                df_sig['60bMed'].plot(ax = ax,label = dev_dets.loc[i,'Participant'])

ax.set_yticks([40,60,80,100,120,140,160,180,200,220,240])
ax.set_xticks(race_dets['Start time'])
ax.set_xticklabels(race_dets['Start time'].dt.strftime('%H:%M'))
ax.set_xlim([t1,t2])
ax.set_ylabel('BPM (60 bts)')
ax.set_title('All participants, BPM')
ax.legend()
ax.grid()
plt.show()           

In [ ]:
V['DateTime']